In [1]:
import sys
sys.path.append('/host/d/Github')
import os
import numpy as np
import pandas as pd
import nibabel as nb
import Osteosarcoma.functions_collection as ff

### Rename the folder and image file

In [2]:
main_path = '/host/d/Data/Habitats/Jishuitan/original_data'

In [ ]:
# # rename folders

# folders = ff.find_all_target_files(['*'], main_path)
# for i in range(0, len(folders)):
#     folder = folders[i]
#     folder_name = os.path.basename(folder)
    
#     # split by '_' and only keep the first part as the new folder name
#     new_folder_name = folder_name.split(' ')[0]
#     new_folder_path = os.path.join(os.path.dirname(folder), new_folder_name)
#     os.rename(folder, new_folder_path)

In [18]:
# rename image files
folders = ff.sort_timeframe(ff.find_all_target_files(['*'], main_path),0)
for i in range(39,40):#, len(folders)):
    folder_name = os.path.basename(folders[i])
    if folder_name == '125' or folder_name == '55':
        continue
    files = ff.find_all_target_files(['*.nii.gz'], folders[i])
    assert len(files) == 2, 'number of files not equal to 2 in folder: '+folders[i]
    with_label = False
    for j in range(0, len(files)):
        if 'label' in files[j]:
            print('label file', files[j])
            with_label = True
            if os.path.basename(files[j]) != 'label .nii.gz':
                new_name = os.path.join(os.path.dirname(files[j]), 'label.nii.gz')
                os.rename(files[j], new_name)
        # else:
        #     image_file = files[j]
        #     new_name = os.path.join(os.path.dirname(image_file), 'img.nii.gz')
        #     os.rename(image_file, new_name)
        
    
    assert with_label == True, 'no label file found in folder: '+folders[i]

label file /host/d/Data/Habitats/Jishuitan/original_data/66/label.nii.gz


### check the data
(1) whether the dimension of img and label are the same  
(2) whether the spacing of img and label are the same  
(3) whether there is only 1 foreground class in the segmentation

(4) whether the slices with segmentation are continuous


In [22]:
main_path = '/host/d/Data/Habitats/Jishuitan/original_data'
folders = ff.sort_timeframe(ff.find_all_target_files(['*'], main_path),0)
for i in range(0, len(folders)):

    folder_name = os.path.basename(folders[i])
    img_file = os.path.join(folders[i], 'img.nii.gz')
    label_file = os.path.join(folders[i], 'label.nii.gz')
    img_nib = nb.load(img_file)
    label_nib = nb.load(label_file)

    # check spacing
    img_spacing = img_nib.header.get_zooms()
    label_spacing = label_nib.header.get_zooms()
    assert img_spacing == label_spacing, 'spacing not equal in folder: '+folders[i]

    # check dimension
    img_data = img_nib.get_fdata()
    label_data = label_nib.get_fdata()
    assert img_data.shape == label_data.shape, 'dimension not equal in folder: '+folders

    # check number of classes
    unique_labels = np.unique(label_data)
    assert len(unique_labels) == 2, 'number of classes not equal to 2 in folder: '+folders[i]

    # check continuity of slices with segmentation
    seg_slices = np.where(np.sum(np.sum(label_data, axis=0), axis=0) > 0)[0]
    slice_diffs = np.diff(seg_slices)
    discontinuities = np.where(slice_diffs > 1)[0]
    if len(discontinuities) >0:
        print('Slices with segmentation are not continuous in folder: '+folders[i])

Slices with segmentation are not continuous in folder: /host/d/Data/Habitats/Jishuitan/original_data/66


In [ ]:
# ### manually correct the discontinuity issues in the following folders
# patient_id = '66'
# img_file = os.path.join(main_path, patient_id, 'img.nii.gz')
# label_file = os.path.join(main_path, patient_id, 'label.nii.gz')
# img_nib = nb.load(img_file)
# affine = img_nib.affine
# img_data = img_nib.get_fdata()
# label_data = nb.load(label_file).get_fdata()

# # find out in the slice range of segmentation, which slices have no segmentation
# seg_slices = np.where(np.sum(np.sum(label_data, axis=0), axis=0) > 0)[0]
# start_slice = seg_slices[0]
# end_slice = seg_slices[-1]
# for slice_idx in range(start_slice, end_slice+1):
#     if np.sum(label_data[:, :, slice_idx]) == 0:
#         print(f'Filling slice {slice_idx} with segmentation')
#         # find the nearest slice with segmentation before and after
#         prev_slice = slice_idx - 1
#         next_slice = slice_idx + 1
#         while prev_slice >= start_slice and np.sum(label_data[:, :, prev_slice]) == 0:
#             prev_slice -= 1
#         while next_slice <= end_slice and np.sum(label_data[:, :, next_slice]) == 0:
#             next_slice += 1
#         # fill the current slice with the average of the two nearest slices
#         if prev_slice >= start_slice and next_slice <= end_slice:
#             label_data[:, :, slice_idx] = (label_data[:, :, prev_slice] + label_data[:, :, next_slice]) / 2
#         elif prev_slice >= start_slice:
#             label_data[:, :, slice_idx] = label_data[:, :, prev_slice]
#         elif next_slice <= end_slice:
#             label_data[:, :, slice_idx] = label_data[:, :, next_slice]

# new_label_nib = nb.Nifti1Image(label_data, affine)
# nb.save(new_label_nib, os.path.join(main_path, patient_id, 'label.nii.gz'))




Filling slice 7 with segmentation


### get a patient list

In [25]:
main_path = '/host/d/Data/Habitats/Jishuitan/original_data'

In [29]:
from xml.etree.ElementInclude import include


label_list = os.path.join('/host/d/Data/Habitats/Jishuitan/Patient_lists', 'labels.xlsx')
df = pd.read_excel(label_list, dtype={'Patient_index': str}).fillna('')
include_list = []
x_shape_list = []
y_shape_list = []
slice_num_list = []
spacing_list = []
img_filepath_list = []
mask_filepath_list = []

for i in range(0, df.shape[0]):
    row = df.iloc[i]
    seg_file = os.path.join(main_path, df['Patient_index'][i], 'label.nii.gz')

    if row['Pathologic_Response_Necrosis_gt90pct'] == '' or os.path.exists(seg_file) == False:
        include_list.append('No')
        x_shape_list.append('')
        y_shape_list.append('')
        slice_num_list.append('')
        spacing_list.append('')
        img_filepath_list.append('')
        mask_filepath_list.append('')
    else:
        include_list.append('Yes')
    
        seg_nib = nb.load(seg_file).get_fdata()
        x_shape_list.append(seg_nib.shape[0])
        y_shape_list.append(seg_nib.shape[1])
        slice_num_list.append(seg_nib.shape[2])
        spaing = nb.load(seg_file).header.get_zooms()
        spacing_list.append(spaing)
        img_filepath_list.append(os.path.join(main_path, df['Patient_index'][i], 'img.nii.gz'))
        mask_filepath_list.append(seg_file)

df['Include'] = include_list
df['X_shape'] = x_shape_list
df['Y_shape'] = y_shape_list
df['Slice_num'] = slice_num_list
df['Spacing'] = spacing_list
df['Image_filepath'] = img_filepath_list
df['Mask_filepath'] = mask_filepath_list
df.to_excel(os.path.join('/host/d/Data/Habitats/Jishuitan/Patient_lists', 'labels_with_image_info.xlsx'), index=False)
